Import library yang dibutuhkan

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, RobustScaler, StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_curve, precision_recall_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import spearmanr
from sklearn.feature_selection import mutual_info_classif
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

akses ke google drive

In [2]:
from google.colab import drive
drive.mount('/drive/', force_remount=True)

Mounted at /drive/


(1) Download dataset pada link :

(2) Upload di drive masing-masing

(3) gunakan data churn_bank.csv, drop kolom yang tidak dibutuhkan.

(4) Define target/output = "churn", feature/input kolom lainnya

(5) Split data menjadi training set dan test set (test_size = 0.2) dengan stratify pada kolom **churn**

(6) print shape pada X_train, X_test, y_train, dan y_test

In [3]:
file_path = '/drive/MyDrive/Colab Notebooks/churn_bank.csv'

In [4]:
df = pd.read_csv(file_path)

In [5]:
df = df.drop('customer_id', axis=1)

In [6]:
print(df.head())

   credit_score country  gender  age  tenure    balance  products_number  \
0           619  France  Female   42       2       0.00                1   
1           608   Spain  Female   41       1   83807.86                1   
2           502  France  Female   42       8  159660.80                3   
3           699  France  Female   39       1       0.00                2   
4           850   Spain  Female   43       2  125510.82                1   

   credit_card  active_member  estimated_salary  churn  
0            1              1         101348.88      1  
1            0              1         112542.58      0  
2            1              0         113931.57      1  
3            0              0          93826.63      0  
4            1              1          79084.10      0  


In [7]:
#Memisahkan fitur (X) dan target (y)
#Menghilangkan kolom 'churn' dari fitur
X = df.drop('churn', axis=1)
#Menggunakan hanya kolom 'churn' sebagai target
y = df['churn']
# Menampilkan beberapa baris pertama dari fitur
print("Fitur (X):")
print(X.head())
# Menampilkan beberapa baris pertama dari target
print("\nTarget (y):")
print(y.head())

Fitur (X):
   credit_score country  gender  age  tenure    balance  products_number  \
0           619  France  Female   42       2       0.00                1   
1           608   Spain  Female   41       1   83807.86                1   
2           502  France  Female   42       8  159660.80                3   
3           699  France  Female   39       1       0.00                2   
4           850   Spain  Female   43       2  125510.82                1   

   credit_card  active_member  estimated_salary  
0            1              1         101348.88  
1            0              1         112542.58  
2            1              0         113931.57  
3            0              0          93826.63  
4            1              1          79084.10  

Target (y):
0    1
1    0
2    1
3    0
4    0
Name: churn, dtype: int64


In [8]:
# Memisahkan data menjadi train set dan test set (test_size = 0.2) dengan stratify pada kolom 'churn'
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Menampilkan bentuk (shape) dari X_train, X_test, y_train, dan y_test
print("Bentuk (shape) dari X_train:", X_train.shape)
print("Bentuk (shape) dari X_test:", X_test.shape)
print("Bentuk (shape) dari y_train:", y_train.shape)
print("Bentuk (shape) dari y_test:", y_test.shape)

Bentuk (shape) dari X_train: (8000, 10)
Bentuk (shape) dari X_test: (2000, 10)
Bentuk (shape) dari y_train: (8000,)
Bentuk (shape) dari y_test: (2000,)


# Imbalance Data Handling

(1) Cek nilai sebaran output/target

(2) lakukan imbalance data handling (oversampling atau undersampling)

(3) fit_resample dan dapatkan dataset baru yang dengan nilai output yang balance

In [9]:
# Melihat sebaran nilai output/target (churn)
print("Sebaran nilai output/target (churn):")
print(y.value_counts())

Sebaran nilai output/target (churn):
0    7963
1    2037
Name: churn, dtype: int64


In [10]:
# Melakukan One-Hot Encoding pada kolom 'country' dan 'gender'
X_train_encoded = pd.get_dummies(X_train, columns=['country', 'gender'])

In [11]:
from imblearn.over_sampling import SMOTE

# Membuat objek SMOTE
smote = SMOTE(random_state=42)

# Melakukan oversampling pada X_train dan y_train
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_encoded, y_train)

# Melihat bentuk (shape) dari X_train_balanced dan y_train_balanced
print("Bentuk (shape) dari X_train_balanced:", X_train_balanced.shape)
print("Bentuk (shape) dari y_train_balanced:", y_train_balanced.shape)

Bentuk (shape) dari X_train_balanced: (12740, 13)
Bentuk (shape) dari y_train_balanced: (12740,)


# Imputation

(1) Cek missing value pada x_train

(2) lakukan imputation dengan menggunakan KNNImputer (jika perlu)

In [12]:
# Cek missing value pada X_train
missing_values = X_train.isnull().sum()
print("Jumlah Missing Value pada Setiap Kolom:")
print(missing_values)

Jumlah Missing Value pada Setiap Kolom:
credit_score        0
country             0
gender              0
age                 0
tenure              0
balance             0
products_number     0
credit_card         0
active_member       0
estimated_salary    0
dtype: int64


In [13]:
# Melakukan One-Hot Encoding pada kolom 'country' dan 'gender'
X_train_encoded = pd.get_dummies(X_train, columns=['country', 'gender'])

In [14]:
from sklearn.impute import KNNImputer

# Membuat objek KNNImputer
imputer = KNNImputer(n_neighbors=5)

# Melakukan imputasi pada X_train_encoded (yang sudah diencode) jika ada missing value
X_train_imputed = imputer.fit_transform(X_train_encoded)

# Mengonversi kembali hasil imputasi ke dalam DataFrame (jika diperlukan)
X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train_encoded.columns)

# Verifikasi apakah masih ada missing value setelah imputasi
print("Jumlah Missing Value pada X_train setelah Imputasi:")
print(X_train_imputed.isnull().sum())

Jumlah Missing Value pada X_train setelah Imputasi:
credit_score        0
age                 0
tenure              0
balance             0
products_number     0
credit_card         0
active_member       0
estimated_salary    0
country_France      0
country_Germany     0
country_Spain       0
gender_Female       0
gender_Male         0
dtype: int64


# Transformation

(1) Lakukan transformation :
- data number pake scalling
- data categorical pake OHE

(2) Pada kolom kategorikal X_train lakukan OHE pada parameter handle_unknown beri argumen 'ignore' agar encoder robust terhadap adanya kategori di testing set yang tidak ada di training set.

In [15]:

# Inisialisasi objek StandardScaler
scaler = StandardScaler()

# Memilih kolom-kolom numerik pada X_train
X_train_numeric = X_train[['credit_score', 'age', 'tenure', 'balance', 'products_number','estimated_salary']]

# Melakukan scaling pada data numerik
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)

In [16]:
# One-Hot Encoding pada data kategorikal
X_train_categorical = X_train[['country', 'gender']]

# Melakukan One-Hot Encoding pada kolom kategorikal dengan handle_unknown='ignore'
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_categorical_encoded = encoder.fit_transform(X_train_categorical).toarray()

# Mengonversi hasil encoding ke dalam DataFrame (jika diperlukan)
X_train_categorical_encoded_df = pd.DataFrame(X_train_categorical_encoded, columns=encoder.get_feature_names_out(X_train_categorical.columns))

In [17]:
from sklearn.preprocessing import OneHotEncoder

# Inisialisasi objek OneHotEncoder dengan handle_unknown='ignore'
encoder = OneHotEncoder(handle_unknown='ignore')

# Memilih kolom kategorikal pada X_train untuk diencode
X_train_categorical_subset = X_train[['country', 'gender']]

# Melakukan One-Hot Encoding dengan handle_unknown='ignore' pada kolom kategorikal
X_train_categorical_encoded = encoder.fit_transform(X_train_categorical_subset).toarray()

# Memeriksa hasil encoding
print(X_train_categorical_encoded_df.head())

   country_France  country_Germany  country_Spain  gender_Female  gender_Male
0             1.0              0.0            0.0            0.0          1.0
1             0.0              1.0            0.0            0.0          1.0
2             0.0              1.0            0.0            1.0          0.0
3             1.0              0.0            0.0            0.0          1.0
4             0.0              1.0            0.0            0.0          1.0


# Feature Selection

(1) Lakukan feature selection menggunakan multicolinearity reduction dan mutual info.

In [18]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Melakukan One-Hot Encoding pada kolom-kolom kategorikal di X_train
X_train_encoded = pd.get_dummies(X_train, columns=['country', 'gender'])

# Menghitung VIF untuk setiap variabel
vif = pd.DataFrame()
vif["Feature"] = X_train_encoded.columns
vif["VIF"] = [variance_inflation_factor(X_train_encoded.values, i) for i in range(X_train_encoded.shape[1])]

# Menampilkan hasil VIF
print(vif)

             Feature       VIF
0       credit_score  1.001041
1                age  1.012386
2             tenure  1.002479
3            balance  1.337182
4    products_number  1.125593
5        credit_card  1.001757
6      active_member  1.010647
7   estimated_salary  1.001499
8     country_France       inf
9    country_Germany       inf
10     country_Spain       inf
11     gender_Female       inf
12       gender_Male       inf


In [19]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Menghitung VIF untuk setiap variabel numerik
vif = pd.DataFrame()
vif["Feature"] = X_train_numeric.columns
vif["VIF"] = [variance_inflation_factor(X_train_numeric.values, i) for i in range(X_train_numeric.shape[1])]

# Pilih fitur dengan VIF rendah
selected_features_vif = vif[vif["VIF"] < 10]["Feature"]

# X_train setelah multicollinearity reduction
X_train_vif = X_train_numeric[selected_features_vif]

In [20]:

from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [21]:

from sklearn.compose import ColumnTransformer

# Mengidentifikasi kolom-kolom kategorikal
categorical_cols = ['country', 'gender']

# Menggunakan ColumnTransformer untuk melakukan One-Hot Encoding pada kolom kategorikal
preprocessor = ColumnTransformer(transformers=[('cat', OneHotEncoder(), categorical_cols)],
                               remainder='passthrough')

# Melakukan transformasi pada X_train
X_train_encoded = preprocessor.fit_transform(X_train)

# Menggunakan mutual information untuk memilih fitur terbaik setelah One-Hot Encoding
selector = SelectKBest(score_func=mutual_info_classif, k='all')
X_train_mi = selector.fit_transform(X_train_encoded, y_train)

# Mendapatkan indeks fitur terbaik
selected_features_mi_indices = selector.get_support(indices=True)

# Memilih fitur dari X_train_encoded menggunakan indeks fitur terbaik
X_train_mi = X_train_encoded[:, selected_features_mi_indices]

# Modelling

(1) Buatlah model Linear Regression, Decision Tree, Random Forest, dan XGBoost

(2) Lakukan fitting untuk training dataset

(3) Buat sebuah function classif yang bisa mencetak nilai metrics evaluation

(4) Lakukan evaluasi pada masing-masing model

In [22]:

from xgboost import XGBClassifier

In [23]:
# Tentukan model Logistic Regression
logreg_model = LogisticRegression()


In [24]:
from sklearn.pipeline import Pipeline

In [25]:
# Gabungkan preprocessor dan model ke dalam Pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', logreg_model)])


In [26]:
# Tentukan parameter grid untuk Grid Search
param_grid = {
    'preprocessor__cat__onehot__drop': [None, 'first'],
    'classifier__C': [0.1, 1, 10, 100]
}

In [72]:
# Gunakan GridSearchCV untuk mencari parameter terbaik
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['country',
                                                                          'gender'])])),
                                       ('classifier',
                                        RandomForestClassifier(random_state=42))]),
             param_grid={'classifier__max_depth': [10, 20, 30],
                         'classifier__min_samples_split': [2, 5, 10],
                         'classifier__n_estimators': [100, 200, 300]},
             scoring='accuracy')

In [63]:
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

Best Parameters: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 100}


In [62]:
# Latih model dengan parameter terbaik
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['country', 'gender'])])),
                ('classifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_c...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=3, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=100, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [71]:
# Evaluasi model di atas data uji
y_pred = best_model.predict(X_test)
print("Classification Report on Test Data:")
print(classification_report(y_test, y_pred))

Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89      1607
           1       0.00      0.00      0.00       393

    accuracy                           0.80      2000
   macro avg       0.40      0.50      0.45      2000
weighted avg       0.65      0.80      0.72      2000



In [31]:

# Inisialisasi model
decision_tree_model = DecisionTreeClassifier(random_state=42)
random_forest_model = RandomForestClassifier(random_state=42)
xgboost_model = XGBClassifier(random_state=42)

# Daftar model
models = {
    "Decision Tree": decision_tree_model,
    "Random Forest": random_forest_model,
    "XGBoost": xgboost_model
}

In [32]:
# Membuat transformer untuk kategori
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Menggabungkan transformer kategori dengan transformer lainnya (jika ada)
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols)
        # (Tambahkan transformer lain jika diperlukan)
    ],
    remainder='passthrough'
)

# Menyiapkan pipeline untuk preprocessing data
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier(random_state=42))])

# Melatih model dengan pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['country', 'gender'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def classif(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")

In [34]:
# Pisahkan data menjadi data pelatihan dan data uji (contoh: menggunakan 20% data untuk uji)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi objek OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Terapkan One-Hot Encoding pada data pelatihan dan data uji
X_train_encoded = encoder.fit_transform(X_train[categorical_cols])
X_test_encoded = encoder.transform(X_test[categorical_cols])

# Ambil kolom-kolom numerik dari data pelatihan dan data uji
X_train_numeric = X_train.drop(categorical_cols, axis=1)
X_test_numeric = X_test.drop(categorical_cols, axis=1)

# Gabungkan data numerik dengan data yang sudah diencode
X_train_encoded = np.hstack((X_train_numeric, X_train_encoded))
X_test_encoded = np.hstack((X_test_numeric, X_test_encoded))


In [60]:

label_encoders = {}
for column in ['country', 'gender']:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])

# Memisahkan fitur (X) dan target (y)
X = df.drop('churn', axis=1)
y = df['churn']

# Membagi data menjadi data pelatihan dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi model Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Melatih model dengan data pelatihan
rf_model.fit(X_train, y_train)

# Menggunakan model untuk membuat prediksi pada data uji
y_pred_class = rf_model.predict(X_test)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred_class)

# Menghitung presisi
precision = precision_score(y_test, y_pred_class)

# Menghitung recall
recall = recall_score(y_test, y_pred_class)

# Menghitung F1-score
f1 = f1_score(y_test, y_pred_class)

# Menampilkan hasil evaluasi
print("Model Random Forest")
print("Akurasi: {:.2f}".format(accuracy))
print("Presisi: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1-Score: {:.2f}".format(f1))


Model Random Forest
Akurasi: 0.86
Presisi: 0.75
Recall: 0.47
F1-Score: 0.57


In [36]:
# Membuat transformer untuk kategori
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Menggabungkan transformer kategori dengan transformer lainnya (jika ada)
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols),
        # (Tambahkan transformer lain jika diperlukan)
    ],
    remainder='passthrough'
)

# Menyiapkan pipeline untuk preprocessing data dan melatih model Decision Tree
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', DecisionTreeClassifier(random_state=42))])

# Latih model dengan data pelatihan
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['country', 'gender'])])),
                ('classifier', DecisionTreeClassifier(random_state=42))])

In [37]:
# Transformasi data pelatihan menggunakan preprocessor
X_train_encoded = preprocessor.fit_transform(X_train)

# Melatih model dengan data pelatihan yang telah diencode
decision_tree_model.fit(X_train_encoded, y_train)

# Transformasi data uji menggunakan preprocessor
X_test_encoded = preprocessor.transform(X_test)

# Menggunakan model untuk membuat prediksi pada data uji yang telah diencode
y_pred_decision_tree = decision_tree_model.predict(X_test_encoded)

In [38]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Menghitung metrik evaluasi
accuracy = accuracy_score(y_test, y_pred_decision_tree)
precision = precision_score(y_test, y_pred_decision_tree)
recall = recall_score(y_test, y_pred_decision_tree)
f1 = f1_score(y_test, y_pred_decision_tree)
conf_matrix = confusion_matrix(y_test, y_pred_decision_tree)

# Menampilkan hasil evaluasi
print("model Decision Tree")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")
print("Confusion Matrix:")
print(conf_matrix)

model Decision Tree
Accuracy: 0.78
Precision: 0.44
Recall: 0.51
F1-Score: 0.47
Confusion Matrix:
[[1354  253]
 [ 193  200]]


In [59]:
label_encoders = {}
for column in ['country', 'gender']:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])

# Memisahkan fitur (X) dan target (y)
X = df.drop('churn', axis=1)
y = df['churn']

# Membagi data menjadi data pelatihan dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi model XGBoost
xgb_model = XGBClassifier()

# Melatih model dengan data pelatihan
xgb_model.fit(X_train, y_train)

# Menggunakan model untuk membuat prediksi pada data uji
y_pred_class = xgb_model.predict(X_test)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred_class)

# Menghitung presisi
precision = precision_score(y_test, y_pred_class)

# Menghitung recall
recall = recall_score(y_test, y_pred_class)

# Menghitung F1-score
f1 = f1_score(y_test, y_pred_class)

# Menampilkan hasil evaluasi
print ("Model Xgboost")
print("Akurasi: {:.2f}".format(accuracy))
print("Presisi: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1-Score: {:.2f}".format(f1))


Model Xgboost
Akurasi: 0.86
Presisi: 0.70
Recall: 0.51
F1-Score: 0.59


# Cross-Validation

(1) Lakukan validation pada masing-masing model. (Terserah mau pake GSCV, secara sequential. atau bayesian optimization)

(2) Gunakan parameter yang diperoleh dari proses validasi untuk membuat model optimasi.

(3) Bandingkan metrics evaluasi dengan model yang sebelumnya. Model mana yang mendapatkan performa terbaik? Berikan analisis kalian

In [40]:
from sklearn.model_selection import GridSearchCV


In [41]:
decision_tree_model = DecisionTreeClassifier()
decision_tree_params = {'criterion': ['gini', 'entropy'], 'max_depth': [None, 10, 20]}

In [42]:
random_forest_model = RandomForestClassifier()
random_forest_params = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}

In [43]:
xgboost_model = XGBClassifier()
xgboost_params = {'n_estimators': [50, 100, 200], 'max_depth': [3, 4, 5], 'learning_rate': [0.1, 0.01]}

In [44]:
# Menggunakan One-Hot Encoding untuk kolom-kolom kategorikal
X_train_encoded = pd.get_dummies(X_train, columns=['country', 'gender'])
X_test_encoded = pd.get_dummies(X_test, columns=['country', 'gender'])


In [45]:
print(X_train)
print(y_train)

      credit_score  country  gender  age  tenure    balance  products_number  \
9254           686   France    Male   32       6       0.00                2   
1561           632  Germany    Male   42       4  119624.60                2   
1670           559    Spain    Male   24       3  114739.92                1   
6087           561   France  Female   27       9  135637.00                1   
6669           517   France    Male   56       9  142147.32                1   
...            ...      ...     ...  ...     ...        ...              ...   
5734           768   France    Male   54       8   69712.74                1   
5191           682   France  Female   58       1       0.00                1   
5390           735   France  Female   38       1       0.00                3   
860            667   France    Male   43       8  190227.46                1   
7270           697  Germany    Male   51       1  147910.30                1   

      credit_card  active_member  estim

In [46]:
print (df.columns)

Index(['credit_score', 'country', 'gender', 'age', 'tenure', 'balance',
       'products_number', 'credit_card', 'active_member', 'estimated_salary',
       'churn'],
      dtype='object')


In [74]:
from sklearn.model_selection import cross_val_score

In [69]:
label_encoders = {}
for column in ['country', 'gender']:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])

In [75]:
# Memisahkan fitur (X) dan target (y)
X = df.drop('churn', axis=1)
y = df['churn']

# Inisialisasi model Logistic Regression
logreg_model = LogisticRegression(random_state=42)

# Lakukan cross-validation dengan 5 fold
cv_scores = cross_val_score(logreg_model, X, y, cv=5, scoring='accuracy')

# Menampilkan hasil cross-validation
print("Cross Validation 'Logistic Regression'")
print("Akurasi setiap fold:", cv_scores)
print("Rata-rata akurasi:", cv_scores.mean())


Cross Validation 'Logistic Regression'
Akurasi setiap fold: [0.7915 0.791  0.793  0.789  0.7885]
Rata-rata akurasi: 0.7906000000000001


In [49]:
decision_tree_model = DecisionTreeClassifier()

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

gscv_decision_tree = GridSearchCV(estimator=decision_tree_model, param_grid=param_grid, scoring='accuracy', cv=5)
gscv_decision_tree.fit(X_train_encoded, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30],
                         'min_samples_split': [2, 5, 10]},
             scoring='accuracy')

In [50]:
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Inisialisasi Label Encoder
label_encoder = LabelEncoder()


# Inisialisasi model Decision Tree
decision_tree_model = DecisionTreeClassifier(random_state=42)

# Tentukan hyperparameter yang akan diuji
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Inisialisasi GSCV dengan model Decision Tree dan parameter grid
gscv_decision_tree = GridSearchCV(estimator=decision_tree_model, param_grid=param_grid, scoring='accuracy', cv=5)

# Melatih GSCV dengan data pelatihan yang telah di-encode
gscv_decision_tree.fit(X_train_encoded, y_train)

# Mendapatkan parameter terbaik
best_params = gscv_decision_tree.best_params_
print("Best Parameters:", best_params)

# Melakukan prediksi menggunakan model terbaik
y_pred_decision_tree = gscv_decision_tree.predict(X_test_encoded)

# Evaluasi model menggunakan akurasi
accuracy_decision_tree = accuracy_score(y_test, y_pred_decision_tree)
print("Accuracy using Decision Tree Classifier (after GSCV, One-Hot Encoding, and Label Encoding):", accuracy_decision_tree)

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10}
Accuracy using Decision Tree Classifier (after GSCV, One-Hot Encoding, and Label Encoding): 0.8355


In [51]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

random_forest_model = RandomForestClassifier(random_state=42)

In [52]:
gscv_random_forest = GridSearchCV(estimator=random_forest_model, param_grid=param_grid, scoring='accuracy', cv=5)

In [53]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', gscv_random_forest)])

In [54]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', random_forest_model)])

In [55]:
# Ubah fitur kategorikal menggunakan one-hot encoding
X_train_encoded = pd.get_dummies(X_train, columns=['country', 'gender'])

# Inisialisasi dan latih model Random Forest dengan data yang telah diproses
gscv_random_forest.fit(X_train_encoded, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy')

In [66]:
# Persiapkan data
X =df.drop('churn', axis=1)
y =df['churn']

# Encode fitur kategorikal
categorical_features = ['country', 'gender']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ])

# Definisikan model
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', RandomForestClassifier())])

# Tentukan parameter grid yang ingin diuji
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],}

# Gunakan GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

# Lakukan pencarian parameter terbaik
grid_search.fit(X, y)

# Tampilkan parameter terbaik dan skor terbaik
print("Parameter Terbaik:", grid_search.best_params_)
print("Akurasi Terbaik:", grid_search.best_score_)

Parameter Terbaik: {'classifier__max_depth': 10, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 100}
Akurasi Terbaik: 0.7963


In [65]:
# Persiapkan data
X = df.drop('churn', axis=1)
y = df['churn']

# Encode fitur kategorikal
categorical_features = ['country', 'gender']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ])

# Definisikan model XGBoost
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', XGBClassifier())])

# Tentukan parameter grid yang ingin diuji
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [3, 4, 5],
    'classifier__learning_rate': [0.1, 0.01, 0.05],
    # Tambahkan parameter lain yang ingin Anda coba di sini
}

# Gunakan GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

# Lakukan pencarian parameter terbaik
grid_search.fit(X, y)

# Tampilkan parameter terbaik dan skor terbaik
print("Parameter Terbaik:", grid_search.best_params_)
print("Akurasi Terbaik:", grid_search.best_score_)

Parameter Terbaik: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 100}
Akurasi Terbaik: 0.7963
